## P7

---

In [ ]:
import numpy as np
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import (
    DataDriftPreset,
    # DataQualityPreset,
    # TargetDriftPreset,
)

%load_ext rich


In [ ]:
import joblib

# load data
app_train_domain = joblib.load("../../data/processed/app_train_domain.joblib")
logistic_y_train = joblib.load("../../data/processed/logistic_y_train.joblib")
logistic_y_test = joblib.load("../../data/processed/logistic_y_test.joblib")
logistic_y_pred = joblib.load("../../data/processed/logistic_y_pred.joblib")
app_test_domain = joblib.load("../../data/processed/app_test_domain.joblib")

logistic_y_pred = pd.DataFrame(logistic_y_pred[:, 1], columns=["y_pred"])
logistic_y_test = pd.DataFrame(logistic_y_test)
logistic_y_train = pd.DataFrame(logistic_y_train)

In [ ]:
# set up data for evidently
categorical_var = app_train_domain.select_dtypes(include=["object"]).columns
numerical_var = app_train_domain.columns.difference(categorical_var)

column_mapping = ColumnMapping(
    target="y_test",
    prediction="y_pred",
    numerical_features=numerical_var.to_list(),
    categorical_features=categorical_var.to_list(),
)
sample_train = app_train_domain.sample(n=100000, random_state=1)
sample_test = app_test_domain.sample(n=10000, random_state=1)

sample_train = sample_train.drop("TARGET", axis=1)

In [ ]:
# Create Data Drift Report
data_drift_report = Report(
    metrics=[
        DataDriftPreset(),
    ]
)


# Step 5: Run the report
data_drift_report.run(
    reference_data=sample_train,
    current_data=sample_test,
    column_mapping=column_mapping,
)

data_drift_report.save_html("../../figures/data_drift_report.html")

In [ ]:
data_drift_report.show()